In [1]:
from ipynb.fs.full.setup import *
from ipynb.fs.full.A_matrix import *
from ipynb.fs.full.boundaryCellDisplacement import *

40
5
40
Shear Modulus (mu): 76923076923.07692
Lame Modulus (Lambda): 115384615384.61539


In [2]:
# Write a function that reads each corner edge BC (traction or displacement) and applies the correct a and b terms

#This will take in edges array, example: [t,l] -> this means top left corner
#It will then search BC_settings 

def cell_corner_BCs(A_matrix, b_matrix, k, edges, xy, U_previous, U_old, U_old_old):

    a_N = A(xy).a_N
    a_S = A(xy).a_S
    a_E = A(xy).a_E
    a_W = A(xy).a_W

    #Overrule a terms for on the boundary

    for edge in edges:

        if edge == "t":

            if BC_settings("t").traction:

                # Apply traction settings to North term
                a_N = 0        
            
            elif BC_settings("t").fixed_displacement:

                # Apply fixed boundary setings to North term
                a_N = boundaryCellDisplacement(["t"], xy).a_N        

        if edge == "b":

            if BC_settings("b").traction:

                # Apply traction settings to South term
                a_S = 0

            elif BC_settings("b").fixed_displacement:

                # Apply fixed boundary setings to South term
                a_S = boundaryCellDisplacement(["b"], xy).a_S

        if edge == "r":

            if BC_settings("r").traction:

                # Apply traction settings to East term
                a_E = 0

            elif BC_settings("r").fixed_displacement:

                # Apply fixed boundary setings to East term
                a_E = boundaryCellDisplacement(["r"], xy).a_E

        if edge == "l":

            if BC_settings("l").traction:

                # Apply traction settings to West term
                a_W = 0

            elif BC_settings("l").fixed_displacement:

                # Apply fixed boundary setings to West term
                a_W = boundaryCellDisplacement([edge], xy).a_W

    if transient:
        a_P = (rho*dx*dy/(dt**2)) + a_N + a_S + a_E + a_W

    else:
        a_P = a_N + a_S + a_E + a_W

    #Aply a terms to matrices

    A_matrix[k,k] = a_P

    #an
    A_matrix[k,index(k).n] = - a_N        
    #as
    A_matrix[k,index(k).s] = - a_S   
    #ae
    A_matrix[k, index(k).e] = - a_E
    #aw
    A_matrix[k, index(k).w] = - a_W


    # b diff

    def b_diff(edges, k, xy, U_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        N_term =(
                    Sfy*A.coef(xy, "N", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "NW", uv, U_previous, k))
                    /dx)
                )
        S_term =(
                    Sfy*A.coef(xy, "S", uv)*(
                        (boundaryCellDisplacement.corner(edges, "SE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, U_previous, k))
                        /dx)
                ) 
        E_term =(
                    Sfx*A.coef(xy, "E", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SE", uv, U_previous, k))
                        /dy)
                ) 
        W_term =(
                    Sfx*A.coef(xy, "W", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NW", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, U_previous, k))
                        /dy)
                )

        for edge in edges:

            if BC_settings("t").traction:
                if (edge == "t") & (xy == "x") : N_term =  Sfy*tr_top_x  
                if (edge == "t") & (xy == "y") : N_term =  Sfy*tr_top_y 

            if BC_settings("b").traction:
                if (edge == "b") & (xy == "x") : S_term =  Sfy*tr_bottom_x  
                if (edge == "b") & (xy == "y") : S_term =  Sfy*tr_bottom_y 

            if BC_settings("r").traction:
                if (edge == "r") & (xy == "x") : E_term =  Sfx*tr_right_x  
                if (edge == "r") & (xy == "y") : E_term =  Sfx*tr_right_y     

            if BC_settings("l").traction:
                if (edge == "l") & (xy == "x") : W_term =  Sfx*tr_left_x  
                if (edge == "l") & (xy == "y") : W_term =  Sfx*tr_left_y      

        b_diffusion = N_term + S_term + E_term + W_term

        return b_diffusion


    b_matrix[k] =(
            A.b_temp(U_old, U_old_old, k, xy)
            +
            b_diff(edges, k, xy, U_previous)
        )  

    return A_matrix, b_matrix    

